In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from dotenv import load_dotenv

load_dotenv("../.env", override=True)

False

In [3]:
from sou.reasoning.run_reasoning import run_reasoning_loop
from sou.reasoning.config import ReasoningSettings

settings = ReasoningSettings(
    model_name="groq/meta-llama/llama-4-scout-17b-16e-instruct",
)


ModuleNotFoundError: No module named 'sou'

In [4]:
prompt = "What is llama4? Do you think its score in benchmark is like presented in the technical report of meta team or is it maybe a scam? "
final_sequence = run_reasoning_loop(prompt, reasoning_settings=settings)

Token indices sequence length is longer than the specified maximum sequence length for this model (1151 > 1024). Running this sequence through the model will result in indexing errors
08:36:25 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= meta-llama/llama-4-scout-17b-16e-instruct; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/llama-4-scout-17b-16e-instruct; provider = groq


Forcing search: False
===========token count 1151
Prompt : 
 You are a reasoning assistant with the ability to perform web searches and execute code to help you answer the user's question accurately. You have special tools:

- To perform a search: write <begin_search_query> your query here </end_search_query>.
Then, the system will search and analyze relevant web pages, then provide you with helpful information in the format <begin_search_result> ...search results... </end_search_result>.
Make sure your each search query is clear and concise.

- To perform calculations or data processing, you can propose a code task using: <begin_code_query> your code query here </end_code_query>.
The system will write the code and execute the code then return the results in the format <begin_code_result> ...execution results... </end_code_result>.
Make sure your each code query is self-contained and does not require any external information.

- To access your reasoning memory, you can query the automa

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
08:36:27 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
08:36:27 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: groq/meta-llama/llama-4-scout-17b-16e-instruct
INFO:LiteLLM:selected model name for cost calculation: groq/meta-llama/llama-4-scout-17b-16e-instruct
08:36:27 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: groq/meta-llama/llama-4-scout-17b-16e-instruct
INFO:LiteLLM:selected model name for cost calculation: groq/meta-llama/llama-4-scout-17b-16e-instruct
08:36:27 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: meta-llama/llama-4-scout-17b-16e-instruct
INFO:LiteLLM:selected model name for cost calculation: meta-llama/llama-4-scout-17b-16e-instruct
08:36:27 - LiteLLM:INFO: cost_calculato

Response : 
 To answer whether llama4 is a scam or its benchmark score is legitimate as presented in the technical report by the Meta team, I need to gather information about llama4, its benchmark scores, and any related technical reports from Meta.

First, I will search for information on llama4.

Assistant:
<begin_search_query>What is llama4?
Search query: What is llama4? Code query: None Rag query: None
Running Search Agent
[SearchAgent] Gathering information for query: What is llama4?
[SearchAgent] Analyzing data:
['Meta has just released Llama 4, their next-generation large language model family. Meta Llama 4 is the latest generation of large language models from Meta. The new Llama 4 models are Meta’s first models that use a mixture of experts (MoE) architecture. Meta’s Llama Scout a powerful yet efficient multimodal large language model. Meta states that the 17 billion active parameter model has 16 experts and is the ‘best multimodal model in the world in its class [..] is more 

Token indices sequence length is longer than the specified maximum sequence length for this model (1069 > 1024). Running this sequence through the model will result in indexing errors
08:36:31 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= llama3-70b-8192; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= llama3-70b-8192; provider = groq


...

Response : 
 Thought: Based on the information gathered, it appears that Llama4, specifically Llama4 Maverick, has achieved impressive benchmark scores, including an ELO score of 1,417 on the Chatbot Arena benchmark. However, there are allegations that Meta may have manipulated the benchmark scores by training the models on test sets or submitting modified versions of the models for performance benchmarks. Independent reviews and real-world performance metrics suggest that the actual capabilities of Llama4 may not match the claimed benchmark scores.

To provide a comprehensive conclusion, I will summarize the findings:

1. **Llama4 Overview**: Llama4 is the latest generation of large language models from Meta, featuring a mixture of experts (MoE) architecture. It includes powerful multimodal models designed to be efficient and powerful.

2. **Benchmark Scores**: Llama4 achieved impressive benchmark scores, including an ELO score of 1,417 on the Chatbot Arena benchmark.

3. **Allegation

08:36:46 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= meta-llama/llama-4-scout-17b-16e-instruct; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/llama-4-scout-17b-16e-instruct; provider = groq


===========token count 556
Prompt : 
 
    Given the following context:
    
    '''
    Thought: Based on the information gathered, it appears that Llama4, specifically Llama4 Maverick, has achieved impressive benchmark scores, including an ELO score of 1,417 on the Chatbot Arena benchmark. However, there are allegations that Meta may have manipulated the benchmark scores by training the models on test sets or submitting modified versions of the models for performance benchmarks. Independent reviews and real-world performance metrics suggest that the actual capabilities of Llama4 may not match the claimed benchmark scores.

To provide a comprehensive conclusion, I will summarize the findings:

1. **Llama4 Overview**: Llama4 is the latest generation of large language models from Meta, featuring a mixture of experts (MoE) architecture. It includes powerful multimodal models designed to be efficient and powerful.

2. **Benchmark Scores**: Llama4 achieved impressive benchmark scores, incl

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
08:36:46 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
08:36:46 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: groq/meta-llama/llama-4-scout-17b-16e-instruct
INFO:LiteLLM:selected model name for cost calculation: groq/meta-llama/llama-4-scout-17b-16e-instruct
08:36:46 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: groq/meta-llama/llama-4-scout-17b-16e-instruct
INFO:LiteLLM:selected model name for cost calculation: groq/meta-llama/llama-4-scout-17b-16e-instruct
08:36:46 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: meta-llama/llama-4-scout-17b-16e-instruct
INFO:LiteLLM:selected model name for cost calculation: meta-llama/llama-4-scout-17b-16e-instruct
08:36:46 - LiteLLM:INFO: cost_calculato

Response : 
 Llama4 is not a scam but an advanced AI model developed by Meta, featuring a mixture of experts (MoE) architecture and powerful multimodal models designed to be efficient and powerful. It has achieved impressive benchmark scores, including an ELO score of 1,417 on the Chatbot Arena benchmark.

However, there are legitimate concerns about the accuracy and legitimacy of these scores as presented by Meta. Some critics argue that the company may have manipulated the benchmarks, either by training the models on test sets or submitting modified versions of the models for performance benchmarks.

Independent reviews and real-world performance metrics suggest that Llama4's actual capabilities may not match the claimed benchmark scores, particularly in coding tasks. Therefore, while Llama4 appears to be a real and technologically advanced model, its benchmark scores might not entirely reflect its real-world performance.

In conclusion, Llama4 is likely not a scam in the traditional

In [5]:
print(final_sequence.final_result)

Llama4 is not a scam but an advanced AI model developed by Meta, featuring a mixture of experts (MoE) architecture and powerful multimodal models designed to be efficient and powerful. It has achieved impressive benchmark scores, including an ELO score of 1,417 on the Chatbot Arena benchmark.

However, there are legitimate concerns about the accuracy and legitimacy of these scores as presented by Meta. Some critics argue that the company may have manipulated the benchmarks, either by training the models on test sets or submitting modified versions of the models for performance benchmarks.

Independent reviews and real-world performance metrics suggest that Llama4's actual capabilities may not match the claimed benchmark scores, particularly in coding tasks. Therefore, while Llama4 appears to be a real and technologically advanced model, its benchmark scores might not entirely reflect its real-world performance.

In conclusion, Llama4 is likely not a scam in the traditional sense, but i

In [6]:
for i in final_sequence.history:
    print(i)

To answer whether llama4 is a scam or its benchmark score is legitimate as presented in the technical report by the Meta team, I need to gather information about llama4, its benchmark scores, and any related technical reports from Meta.

First, I will search for information on llama4.

Assistant:
<begin_search_query>What is llama4?</end_search_query>

<begin_search_result>Llama 4 is the latest generation of large language models from Meta, featuring a mixture of experts (MoE) architecture. It includes powerful multimodal models such as Llama Scout and Llama 4 Maverick, which are designed to be efficient and powerful. These models can be accessed directly from Meta or through Hugging Face.</end_search_result>


Thought: Now that I have a basic understanding of what llama4 is, I need to gather more information about its benchmark scores and compare them with the technical report from the Meta team. This will help determine if the scores are legitimate or if llama4 might be a scam.

Next,

In [7]:
final_sequence.output

"Thought: Based on the information gathered, it appears that Llama4, specifically Llama4 Maverick, has achieved impressive benchmark scores, including an ELO score of 1,417 on the Chatbot Arena benchmark. However, there are allegations that Meta may have manipulated the benchmark scores by training the models on test sets or submitting modified versions of the models for performance benchmarks. Independent reviews and real-world performance metrics suggest that the actual capabilities of Llama4 may not match the claimed benchmark scores.\n\nTo provide a comprehensive conclusion, I will summarize the findings:\n\n1. **Llama4 Overview**: Llama4 is the latest generation of large language models from Meta, featuring a mixture of experts (MoE) architecture. It includes powerful multimodal models designed to be efficient and powerful.\n\n2. **Benchmark Scores**: Llama4 achieved impressive benchmark scores, including an ELO score of 1,417 on the Chatbot Arena benchmark.\n\n3. **Allegations of